# Setup

In [1]:
import sys
# Python 3.7 is required
assert sys.version_info >= (3,7)

import time

import cv2 as cv
import numpy as np

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

# Question 1

In [2]:
def crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color):
    # img is the source image
    # num_horizontal_grid and num_vertical_grid are the number of patches along x and y axes.
    # line_color is the color of the grid line.
    # The output of the function should be image with grids
    img_copy = img.copy()
    
    height, width = img.shape[:2]
    
    M, N = int(height/num_horizontal_grid), int(width/num_vertical_grid)
    x1, y1 = 0, 0

    for y in range(0, height, M):
        for x in range(0, width, N):
    #         if (height - y) < M or (width - x) < N:
    #             break

            y1 = y + M    # lower right coordinate that will be used to construct rectangle
            x1 = x + N

            # Check whether patch lower right coordinate exceeds image height and width
            if x1 >= width and y1 >= height:
                x1 = width - 1
                y1 = height - 1
                tile = img[y:height, x:width]
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
            # When patch lower right y-coordinate exceeds patch height
            elif y1 >= height:
                y1 = height - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)

            # When patch lower right x-coordinate exceeds patch width
            elif x1 >= width:
                x1 = width - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)

            else:
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
    
    cv.imshow('patched image', img_copy)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
img = cv.imread('dog.jfif')
crop_grid(img, 2, 4, (0,255,0))

# Question 2

In [4]:
img1 = cv.imread('lena.jfif')
img2 = cv.imread('coins.jfif')

# Resize img2
new_shape = img1.shape[:2]
img2 = cv.resize(img2, new_shape)

for i in range (0, 13):
    fadein = i / 10.0
    dst = cv.addWeighted(img1, fadein, img2, 1 - fadein, 0)
    cv.imshow('blend_image', dst)
    time.sleep(0.05)
    cv.waitKey(200)
    cv.destroyAllWindows()

# Question 3

## Method 1

In [10]:
import imutils

img = cv.imread('lena.jfif')

rotate_image = imutils.rotate_bound(img, 45)
cv.imshow("rotate image",rotate_image)
cv.waitKey(0)
cv.destroyAllWindows()

## Method 2

In [9]:
img = cv.imread('lena.jfif')

height, width = img.shape[:2] # image shape has 3 dimensions
image_center = (width/2, height/2) # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape

rotation_img = cv.getRotationMatrix2D(image_center, 45, 1)

# rotation calculates the cos and sin, taking absolutes of those.
abs_cos = abs(rotation_img[0,0]) 
abs_sin = abs(rotation_img[0,1])

# find the new width and height
new_w = int(height * abs_sin + width * abs_cos)
new_h = int(height * abs_cos + width * abs_sin)

# subtract old image center and adding the new image center coordinates
rotation_img[0, 2] += new_w/2 - image_center[0]
rotation_img[1, 2] += new_h/2 - image_center[1]

# rotate image with the new bounds and translated rotation matrix
rotation_img = cv.warpAffine(img, rotation_img, (new_w, new_h))

cv.imshow("rotate image",rotation_img)
cv.waitKey(0)
cv.destroyAllWindows()

# Question 4

In [25]:
img1 = cv.imread('flower.jfif')
img2 = cv.imread('native-bee.png')

# get the mask with gray colour of flower
gray_img1 = cv.cvtColor(img1,cv.COLOR_BGR2GRAY)
resized_img = np.zeros((img2.shape[0],img2.shape[1]), dtype = np.uint8)
resized_img[35:120, 85:178] = gray_img1[35:120, 85:178]

# invert the mask to highlight the part we want for flower
ret, thresh = cv.threshold(resized_img, 70, 255, cv.THRESH_BINARY)
inverted_mask = cv.bitwise_not(thresh)

resized_img_rgb = cv.cvtColor(resized_img,cv.COLOR_GRAY2BGR)
resized_img_rgb[35:120, 85:178] = img1[35:120, 85:178]

ROI = cv.bitwise_and(img2, img2, mask = inverted_mask)

flower = cv.bitwise_and(resized_img_rgb, resized_img_rgb, mask = thresh)

# merge both image together
dst = cv.add(ROI,flower)

cv.imshow('bee with flower', dst)
cv.waitKey(0)
cv.destroyAllWindows()